### On a mac, as a docker container

Get docker for your Mac from here: https://store.docker.com/editions/community/docker-ce-desktop-mac

If that link is stale, just Google Docker for Mac.

Install Docker. Make sure it is runnning.

```bash
[Generics-Air] genericuser ~
$ docker --version
Docker version 17.06.2-ce, build cec0b72

[Generics-Air] genericuser ~
$ docker-compose --version
docker-compose version 1.14.0, build c7bdf9e

[Generics-Air] genericuser ~
$ docker run hello-world
Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
5b0f327be733: Pull complete 
Digest: sha256:b2ba691d8aac9e5ac3644c0788e3d3823f9e97f757f01d2ddc6eb5458df9d801
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.
```

Then install the Kafka container as follows:

```bash
[Generics-Air] genericuser ~
$ docker run --rm -it \
              -p 2181:2181 -p 3030:3030 -p 8081:8081 \
              -p 8082:8082 -p 8083:8083 -p 9092:9092 \
              -e ADV_HOST=127.0.0.1 \
               landoop/fast-data-dev
Unable to find image 'landoop/fast-data-dev:latest' locally
latest: Pulling from landoop/fast-data-dev
...
```

When it finishes installation and set up and initialization, you can check that it's working by accessing localhost:3030 on your host Mac.

To login to the bash on your kafka container (once it is up and running), use the following:
```bash
$ docker run --rm -it --net=host landoop/fast-data-dev bash
```

### kafka-topics
```bash
root@fast-data-dev / $ kafka-topics
Create, delete, describe, or change a topic.
.
.
.

root@fast-data-dev / $ kafka-topics --zookeeper 127.0.0.1:2181 --create --topic firsttopic
Missing required argument "[partitions]"
.
.
.

root@fast-data-dev / $ kafka-topics --zookeeper 127.0.0.1:2181 --create --topic firsttopic --partitions 3
Missing required argument "[replication-factor]"
.
.
.

root@fast-data-dev / $ kafka-topics --zookeeper 127.0.0.1:2181 --create --topic firsttopic --partitions 3 --replication-factor 2
Error while executing topic command : replication factor: 2 larger than available brokers: 1
[2017-09-29 19:01:45,748] ERROR org.apache.kafka.common.errors.InvalidReplicationFactorException: replication factor: 2 larger than available brokers: 1
 (kafka.admin.TopicCommand$)
 
root@fast-data-dev / $ kafka-topics --zookeeper 127.0.0.1:2181 --create --topic first_topic --partitions 3 --replication-factor 1
WARNING: Due to limitations in metric names, topics with a period ('.') or underscore ('_') could collide. To avoid issues it is best to use either, but not both.
Created topic "first_topic".

root@fast-data-dev / $ kafka-topics --zookeeper 127.0.0.1:2181 --list
__consumer_offsets
_schemas
backblaze_smart
connect-configs
connect-offsets
connect-statuses
coyote_test_avro
coyote_test_binary
coyote_test_json
first_topic
firsttopic
nyc_yellow_taxi_trip_data
reddit_posts
sea_vessel_position_reports

root@fast-data-dev / $ kafka-topics --zookeeper 127.0.0.1:2181 --create --topic secondtopic --partitions 3 --replication-factor 1
Created topic "secondtopic".

root@fast-data-dev / $ kafka-topics --zookeeper 127.0.0.1:2181 --create --topic secondtopic --partitions 3 --replication-factor 1
Error while executing topic command : Topic 'secondtopic' already exists.
[2017-09-29 19:08:50,192] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'secondtopic' already exists.
 (kafka.admin.TopicCommand$)
 
root@fast-data-dev / $ kafka-topics --zookeeper 127.0.0.1:2181 --topic first_topic --delete
Topic first_topic is marked for deletion.
Note: This will have no impact if delete.topic.enable is not set to true.

root@fast-data-dev / $ kafka-topics --zookeeper 127.0.0.1:2181 --describe --topic firsttopic
Topic:firsttopic	PartitionCount:3	ReplicationFactor:1	Configs:
	Topic: firsttopic	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: firsttopic	Partition: 1	Leader: 0	Replicas: 0	Isr: 0
	Topic: firsttopic	Partition: 2	Leader: 0	Replicas: 0	Isr: 0
```

### kafka-console-producer

```bash
root@fast-data-dev / $ kafka-console-producer --broker-list 127.0.0.1:9092 --topic firsttopic
>hi
>hello
>nikhil
>udemy
>learning about kafka
>first topic again
>new message
```
**Note that since we didn't specify any key, the messages were randomly assigned to any paritition.**

<img src='./kafka-console-producer-1.png'>

<br/><br/>

```bash
root@fast-data-dev / $ kafka-console-producer --broker-list 127.0.0.1:9092 --topic newtopic
>hi
[2017-09-29 19:24:06,825] WARN Error while fetching metadata with correlation id 1 : {newtopic=LEADER_NOT_AVAILABLE} (org.apache.kafka.clients.NetworkClient)
>second message
>interesting... a topic and leader was created automatically  
```
**Note that a new topic was created automatically, with 1 partition and 1 replication**
<img src='./kafka-console-producer-2.png'>

### kafka-console-consumer

- bootstrap-server in consumer is the same as broker-list server in the producer command
- Ignore the zookeeper 'REQUIRED' directive, since it is applicable only for consumers (or producers?) that have an old version... (the idea is that the only time you have to interact zookeeper is when you create a topic)

```bash
<terminal 1>
root@fast-data-dev / $ kafka-console-consumer --bootstrap-server 127.0.0.1:9092 --topic firsttopic
hello from producer
here is a new message
another one
and yet another one
^CProcessed a total of 4 messages

<terminal 2>
[Generics-Air] genericuser ~
$ docker run --rm -it --net=host landoop/fast-data-dev bash
root@fast-data-dev / $ kafka-console-producer --topic firsttopic --broker-list 127.0.0.1:9092
>hello from producer
>here is a new message
>another one
>and yet another one
>^Croot@fast-data-dev / $ kafka-console-producer --topic firsttopic --broker-list 127.0.0.1:9092
>let's do another one
>^Croot@fast-data-dev / $ 

<terminal 1>
root@fast-data-dev / $ kafka-console-consumer --bootstrap-server 127.0.0.1:9092 --topic firsttopic --from-beginning
hello
learning about kafka
another one
nikhil
first topic again
hello from producer
and yet another one
hi
udemy
new message
here is a new message
let's do another one
^CProcessed a total of 12 messages

root@fast-data-dev / $ kafka-console-consumer --bootstrap-server 127.0.0.1:9092 --topic firsttopic --from-beginning --partition 0
hello
learning about kafka
another one
let's do another one
^CProcessed a total of 4 messages

root@fast-data-dev / $ kafka-console-consumer --bootstrap-server 127.0.0.1:9092 --topic firsttopic --from-beginning --partition 1
hi
udemy
new message
here is a new message
^CProcessed a total of 4 messages

root@fast-data-dev / $ kafka-console-consumer --bootstrap-server 127.0.0.1:9092 --topic firsttopic --from-beginning --partition 2
nikhil
first topic again
hello from producer
and yet another one
^CProcessed a total of 4 messages
```

**Notice**
- **how order is maintained only within a partition, and not across it, and how order is kind of mixed when looking across partitions.**
- **how the --from-beginning flag works, and how without it, it starts listening (consuming) only at the tail of the partition**
- **how you can repeatedly consume the same offsets in the same partitions... 'consuming' does not 'destroy' the messages, or change the 'beginning' offset**

```bash
<terminal 1>
root@fast-data-dev / $ kafka-console-consumer --bootstrap-server 127.0.0.1:9092 --topic firsttopic --consumer-property group.id=mygroup1 --from-beginning
hello
learning about kafka
another one
let's do another one
nikhil
first topic again
hello from producer
and yet another one
hi
udemy
new message
here is a new message
^CProcessed a total of 12 messages

<terminal 2>
root@fast-data-dev / $ kafka-console-producer --topic firsttopic --broker-list 127.0.0.1:9092
>hey consumer that's part of a group.id
>message one after consumer was stopped
>message two after
>message three
>message four

<terminal 1>
root@fast-data-dev / $ kafka-console-consumer --bootstrap-server 127.0.0.1:9092 --topic firsttopic --consumer-property group.id=mygroup1 --from-beginning
hey consumer that's part of a group.id
^CProcessed a total of 1 messages
root@fast-data-dev / $ kafka-console-consumer --bootstrap-server 127.0.0.1:9092 --topic firsttopic --consumer-property group.id=mygroup1 --from-beginning
message one after consumer was stopped
message four
message two after
message three
```

**Notice**
- **How being a consumer that's part of the group now causes the 'beginning' offset to be reset after consumption**